In [118]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from collections import defaultdict

## Dicionário de usuários, amigos e interesses

In [117]:
users = [{ "id": 0, "name": "Hero" }, 
         { "id": 1, "name": "Dunn" }, 
         { "id": 2, "name": "Sue" },
         { "id": 3, "name": "Chi" },
         { "id": 4, "name": "Thor" },
         { "id": 5, "name": "Clive" }, 
         { "id": 6, "name": "Hicks" }, 
         { "id": 7, "name": "Devin" }, 
         { "id": 8, "name": "Kate" }, 
         { "id": 9, "name": "Klein" }]

friendships = [(0, 1), (0, 2), (1, 2), (1, 3), 
               (2, 3), (3, 4), (4, 5), (5, 6), 
               (5, 7), (6, 8), (7, 8), (8, 9)]

interests = [(0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"), (0, "Spark"), (0, "Storm"), 
             (0, "Cassandra"), (1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"), (1, "Postgres"), 
             (2, "Python"), (2, "scikit-learn"), (2, "scipy"), (2, "numpy"), (2, "statsmodels"), (2, "pandas"), 
             (3, "R"), (3, "Python"), (3, "statistics"), (3, "regression"), (3, "probability"), 
             (4, "machine learning"), (4, "regression"), (4, "decision trees"), (4, "libsvm"),
             (5, "Python"), (5, "R"), (5, "Java"), (5, "C++"), (5, "Haskell"), (5, "programming languages"),
             (6, "statistics"), (6, "probability"), (6, "mathematics"), (6, "theory"), (7, "machine learning"),
             (7, "scikit-learn"), (7, "Mahout"), (7, "neural networks"), (8, "neural networks"),
             (8, "deep learning"), (8, "Big Data"), (8, "artificial intelligence"), (9, "Hadoop"), 
             (9, "Java"), (9, "MapReduce"), (9, "Big Data")]


## Adição da informação de amizade no dicionário

In [70]:
for user in users:
    user["friends"] = []   #Cria a coluna amigos
    
for i,j in friendships:
    users[i]['friends'].append(users[j]) #Adiciona j como amigo de i
    users[j]['friends'].append(users[i]) #Adiciona i como amigo de j

## Qual o número médio de conexões?

In [71]:
#Número total de conexão por usuário
def number_of_friends(user):
    return len(user["friends"])

total_connections = sum(number_of_friends(user) for user in users)
num_users = len(users)

mean_connections = total_connections/num_users
print("Cada usuário tem em média %.1f amigos" %mean_connections)

Cada usuário tem em média 2.4 amigos


## Qual usuário tem mais amigos?

In [72]:
#cria uma lista (user_id, num_friends)
num_friends_by_id = [(user["id"], number_of_friends(user)) for user in users]

#Ordenação por num_friends
sorted(num_friends_by_id, key = lambda ENTRY: ENTRY[1], reverse = True)

[(1, 3),
 (2, 3),
 (3, 3),
 (5, 3),
 (8, 3),
 (0, 2),
 (4, 2),
 (6, 2),
 (7, 2),
 (9, 1)]

## Contagem de Amigos em Comun (FoaF):

### Forma ruim

In [97]:
#Para cada amigo de um usuário, itera sobre os amigos daquela pessoa, e coleta todos os resultados
def FoaF_id_bad(user):
    return [foaf["id"] 
            for friend in user["friends"]    #Para cada amigo do usuario 'user'
            for foaf in friend["friends"]]  #Para cada amigo do amigo do usuario 'user'

#Amigo dos amigos do usuário id:0
FoaF_id_bad(users[3])

[0, 2, 3, 0, 1, 3, 3, 5]

Essa maneira inclui tanto a propria pessoa caso ela já seja amiga dos amigos dela e também repete as pessoas na lisca caso elas sejam acessíveis por meio de 2 ou mais amigos diferentes. É necessário remover as pessoas já conhecidas e repetidas (vide Forma boa)

### Forma boa

In [116]:
#user2 não é amiga de user1 se ela não está em user1["friends"]
#ou seja, se user2 é not_the_same com ngm de user1["friends"]
def not_friends(user1, user2):
    return all(friend["id"] != user2["id"] for friend in user1["friends"])

def FoaF_id(user):
    return Counter(foaf["id"] 
                   for friend in user["friends"] 
                   for foaf in friend["friends"]
                   if user["id"]!=foaf["id"] and not_friends(user,foaf))
                   
print(FoaF_id(users[3]))

Counter({0: 2, 5: 1})


## Sugestão de usuários com mesmos interesses

In [130]:
# As chaves são interesses, os valores são listas de user_ids com interests
user_id_by_interest = defaultdict(list)
for user_id, interest in interests:
    user_id_by_interest[interest].append(user_id)
    
# As chaves são user_ids, os valores são as listas de interests para aquele user_id
interests_by_user_id = defaultdict(list)
for user_id, interest in interests: 
    interests_by_user_id[user_id].append(interest)
    
def most_common_interests_with(user): 
    return Counter(interested_user_id 
                   for interest in interests_by_user_id[user["id"]]
                   for interested_user_id in user_id_by_interest[interest] if interested_user_id != user["id"])

print(most_common_interests_with(users[3]))

Counter({5: 2, 6: 2, 2: 1, 4: 1})


## Tópicos de maior interesse

In [132]:
words_and_counts = Counter(word 
                           for user, interest in interests 
                           for word in interest.lower().split())

for word, count in words_and_counts.most_common():
    if count > 1:
        print(word, count)

big 3
data 3
java 3
python 3
learning 3
hadoop 2
hbase 2
cassandra 2
scikit-learn 2
r 2
statistics 2
regression 2
probability 2
machine 2
neural 2
networks 2


## 